In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)


2.0.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


In [3]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 864/1000 [========================>.....] - ETA: 0s - loss: 1.2391 - accuracy: 0.6481
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1547 - accuracy: 0.6700 - val_loss: 0.7605 - val_accuracy: 0.7630
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4329 - accuracy: 0.8771
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 478us/sample - loss: 0.4257 - accuracy: 0.8780 - val_loss: 0.5493 - val_accuracy: 0.8240
Epoch 3/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.2921 - accuracy: 0.9163
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 577us/sample - loss: 0.2938 - accuracy: 0.9160 - val_loss: 0.4528 - val_accuracy: 0.8640
Epoch 4/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.2259 - accuracy: 0.9387
Epoch 0

In [5]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


1000/1 - 0s - loss: 2.2163 - accuracy: 0.1660
Untrained model, accuracy: 16.60%


In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1 - 0s - loss: 0.4083 - accuracy: 0.8660
Restored model, accuracy: 86.60%


In [8]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [9]:
!ls {checkpoint_dir}


checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1 - 0s - loss: 0.5473 - accuracy: 0.8680
Restored model, accuracy: 86.80%


In [12]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model shuold be saved to HDF5.
model.save('my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 853us/sample - loss: 1.1682 - accuracy: 0.6560
Epoch 2/5
1000/1000 [==============================] - 0s 352us/sample - loss: 0.4274 - accuracy: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 377us/sample - loss: 0.2864 - accuracy: 0.9200
Epoch 4/5
1000/1000 [==============================] - 0s 302us/sample - loss: 0.2137 - accuracy: 0.9450
Epoch 5/5
1000/1000 [==============================] - 0s 354us/sample - loss: 0.1539 - accuracy: 0.9740


In [13]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [15]:
import tensorflow as tf
import datetime

In [16]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [19]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5,callbacks=[tensorboard_callback])
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 785us/sample - loss: 1.1209 - accuracy: 0.6830
Epoch 2/5
1000/1000 [==============================] - 0s 487us/sample - loss: 0.3986 - accuracy: 0.8870
Epoch 3/5
1000/1000 [==============================] - 0s 420us/sample - loss: 0.2893 - accuracy: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 397us/sample - loss: 0.2055 - accuracy: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 345us/sample - loss: 0.1496 - accuracy: 0.9630


In [18]:
%tensorboard --logdir logs/fit